In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import base64
%matplotlib inline

In [ ]:
authurl = 'https://accounts.spotify.com/api/token'
authheader = {}
authdata = {}

In [ ]:
with open('../keys/client_id.json') as fi:
    credentials = json.load(fi)

In [ ]:
client_id = credentials['client_id']

In [ ]:
with open('../keys/client_secret_id.json') as fi:
    secret_credentials = json.load(fi)

In [ ]:
client_secret_id = secret_credentials['client_secret_id']

In [ ]:
message = f'{client_id}:{client_secret_id}'
message_bytes = message.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')